Schizophrenia Classification

In [3]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import mne
import seaborn as sns
import matplotlib.pyplot as plt

2023-05-30 05:43:54.050534: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 05:43:54.085247: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Data Preprocessing

#### 1.1 Setting paths for data

In [4]:
data_path = "../EEG_Schizophrenia/eeg_data/"

In [5]:
healthy_path = [data_path + file for file in os.listdir(data_path) if file[0] == 'h']
schizo_path = [data_path + file for file in os.listdir(data_path) if file[0] == 's']

In [6]:
print("Number of control group:", len(healthy_path))
print("Number of schizo group:", len(schizo_path))

Number of control group: 14
Number of schizo group: 14


#### 1.2. Creating dataframe

In [7]:
def chunk(seq, size):
    sl = len(seq) - (len(seq)%size) #exclude values that will be out of range
    r = [pd.DataFrame(seq[pos:pos + size]) for pos in range(0, sl, size)]
    return r

In [8]:
import pyedflib

def get_minimum_duration(data_path):
    file_durations = []
    for file in os.listdir(data_path):
        f = pyedflib.EdfReader(data_path + file)
        file_durations.append(f.file_duration)
        f.close()
    return(min(file_durations))

min_duration = get_minimum_duration(data_path)
print(f"Minimum duration length in data: {min_duration}")

Minimum duration length in data: 740


In [9]:
def create_dataframe(file_path):
    file = mne.io.read_raw_edf(file_path, preload=True) # reading edf files from path
    file.set_eeg_reference()                            # calculates reference channel and subtracts it from each channel to reduce noise.
    file.filter(l_freq=1, h_freq=45)                    # bandpass filter(1-45Hz)
    epochs = mne.make_fixed_length_epochs(file, duration=100, overlap=0) #creates fixed length epochs from data.
    epochs_df = epochs.to_data_frame()
    return epochs_df[150: (min_duration - 150)]

In [10]:
healthy_df = pd.DataFrame()
for file in healthy_path:
    df = create_dataframe(file)
    df = df.drop(columns=['time', 'condition', 'epoch'])
    df['PatientNo'] = file.split('.edf')[0][-3:] 
    healthy_df = pd.concat([healthy_df, df]).reset_index(drop=True)

Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/h10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 278749  =      0.000 ...  1114.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
11 matching events found
No baseline correction applied
0 projection items 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/h02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
9 matching events found
No baseline correction applied
0 projection items a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/h08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
9 matching events found
No baseline correction applied
0 projection items a

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
9 matching events found
No baseline correction applied
0 projection items a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/h04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
9 matching events found
No baseline correction applied
0 projection items a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


In [11]:
schizo_df = pd.DataFrame()
for file in schizo_path:
    df = create_dataframe(file)
    df = df.drop(columns=['time', 'condition', 'epoch'])
    df['PatientNo'] = file.split('.edf')[0][-3:] 
    schizo_df = pd.concat([schizo_df, df]).reset_index(drop=True)

Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/s06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 184999  =      0.000 ...   739.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
7 matching events found
No baseline correction applied
0 projection items a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/s05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 222499  =      0.000 ...   889.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
8 matching events found
No baseline correction applied
0 projection items a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
9 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 9 events and 25000 original time points ...
0 bad epochs dropped


...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.2s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/s11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 339999  =      0.000 ...  1359.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
13 matching events found
No baseline correction applied
0 projection items 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Extracting EDF parameters from /home/meliha/Documents/EEG_Schizophrenia/eeg_data/s01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 211249  =      0.000 ...   844.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
8 matching events found
No baseline correction applied
0 projection items a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


In [12]:
healthy_df.head()

,Fp2,F8,T4,T6,O2,Fp1,F7,T3,T5,O1,F4,C4,P4,F3,C3,P3,Fz,Cz,Pz,PatientNo
0,-7.682465,-9.002307,1.759478,6.290979,6.922493,-4.692652,4.470830,1.465356,-3.170504,-0.408481,-8.629520,0.403938,2.504190,0.477834,0.863866,3.945374,-1.710256,-0.573752,6.765600,h10
1,-7.791921,-6.787724,1.554092,6.608817,7.792384,-5.542965,-0.785818,-0.592524,-2.144247,-0.048140,-6.764914,1.111347,3.111187,-0.176970,0.209792,3.830179,-1.023742,0.395254,7.045914,h10
2,-7.674624,-6.716586,1.692419,6.669123,9.002760,-6.399481,-6.076734,-2.356467,-0.902079,0.778253,-5.013057,2.125277,4.148294,-0.658922,-0.147989,3.783223,-0.846660,1.466263,7.126989,h10
3,-6.719919,-8.105915,1.676598,6.609055,9.764501,-6.289082,-8.290625,-3.014580,0.066690,1.428454,-4.088494,2.610536,4.847559,-0.607005,-0.448106,3.378284,-1.200036,1.794705,6.587380,h10
4,-5.077074,-8.201661,1.263723,6.103682,9.321420,-5.189859,-6.599450,-2.569544,0.520866,1.496936,-3.802511,2.307383,4.618692,-0.224713,-0.951879,2.453292,-1.777407,1.010484,5.297621,h10


In [13]:
schizo_df.head()

,Fp2,F8,T4,T6,O2,Fp1,F7,T3,T5,O1,F4,C4,P4,F3,C3,P3,Fz,Cz,Pz,PatientNo
0,13.529647,0.060876,-4.581882,-13.658559,-26.283326,14.757164,16.412265,10.095743,2.354505,-21.269318,4.855470,-2.383202,-16.644509,14.397861,7.901072,-1.665965,9.021134,4.261642,-11.160620,s06
1,14.058557,-0.508339,-5.279422,-18.298712,-28.610756,14.325255,14.925434,9.902323,2.422361,-18.948393,5.775127,-1.282814,-16.595816,14.301865,8.375757,-0.148874,9.782921,5.652707,-9.849182,s06
2,13.404408,-1.298908,-5.562730,-21.027830,-29.417131,13.884697,13.186797,9.723618,2.340198,-16.661986,6.127425,-1.185253,-15.527494,13.492484,8.808401,1.346530,10.012004,6.433874,-8.079103,s06
3,11.277887,-2.575514,-5.594020,-21.744592,-28.855615,12.681523,11.683684,9.826339,2.547925,-13.952907,5.583721,-1.798726,-13.374552,12.078156,9.372154,3.010699,9.425535,6.428714,-6.020413,s06
4,8.351837,-3.992067,-5.403863,-21.060751,-27.341504,10.388137,10.645970,10.089252,3.127170,-10.503924,4.142578,-2.476025,-10.463713,10.337908,9.819218,4.680785,7.844812,5.685891,-3.871711,s06


In [14]:
healthy_array = np.array(healthy_df.drop(columns=['PatientNo']))
schizo_array = np.array(schizo_df.drop(columns=['PatientNo']))

In [15]:
print(np.asarray(healthy_array).shape)
print(np.asarray(schizo_array).shape)

(6160, 19)
(6160, 19)


## 2. Feature Extraction

![Table](./feature_table.png) \
Ref: [Methods of EEG Signal Features Extraction Using Linear Analysis in Frequency and Time-Frequency Domains](https://www.researchgate.net/publication/263477119_Methods_of_EEG_Signal_Features_Extraction_Using_Linear_Analysis_in_Frequency_and_Time-Frequency_Domains)



#### 2.1. FFT

In [151]:
from scipy.fftpack import dct, idct

def fast_fourier_transform(x, dct_type=2):
    fft_dct = dct(dct(x, type=dct_type, norm='ortho'), type=3, norm='ortho') #discreet cosine transform
    fft_idct = idct(dct(x, type=dct_type), type=2)
    return fft_dct, fft_idct
    

#### 2.2. Discrete Wavelet Transform

In [152]:
import pywt

def discrete_wt(X, wavelet_name='db1', mode=None):
    if mode is None:
        r = pywt.dwt(X, wavelet_name)
    else:
        r = pywt.dwt(X, wavelet_name, mode=mode)
    return r


#### 2.3. Continuous Wavelet Transform

In [18]:
from scipy import signal

def continuous_wt(X, show_plot=False, widths=None):
    if 1 in X.shape :
        X = X.flatten()
    if widths is None:
        widths = np.arange(1, len(X) +1)
        widths = np.arange(1, 2)
    sig  = np.cos(2 * np.pi * 7 * X) + signal.gausspulse(X - 0.4, fc=2)
    cwtmatr = signal.cwt(sig, signal.ricker, widths)
    return cwtmatr

#### 2.4. Burg's Method (AR)

In [19]:
def burgs(x, k):

    def sumsq(x):
        return np.sum(x * x)

    n = x.size
    forward_err_seq = x[1:n]
    backward_err_seq = x[:(n - 1)]
    a = np.array([])

    for i in range(k):
        denominator = sumsq(forward_err_seq) + sumsq(backward_err_seq)
        g = 0 if denominator == 0 else 2 * np.sum(forward_err_seq * backward_err_seq) / denominator
        if i == 0:
            a = np.array([g])
        else:
            a = np.append(g, a - g * a[:(i)][::-1])

        old_f = np.empty_like(forward_err_seq)
        old_f[:] = forward_err_seq
        forward_err_seq = old_f[1:(n - i - 1)] - g * backward_err_seq[1:(n - i - 1)]
        backward_err_seq = backward_err_seq[:(n - i - 2)] - g * old_f[:(n - i - 2)]

    return -a[:k][::-1]
    

#### 2.4 Creating feature dataset

In [170]:
def extract_features(data):
    features = []
    for row in data:
        fft, i_fft = fast_fourier_transform(row)
        dwt = np.array(discrete_wt(row))
        dwt.resize(19)
        cwt = continuous_wt(row)[0]
        burg = burgs(row, len(row))
        temp = np.asarray([fft, i_fft, dwt, cwt, burg], dtype=np.float32)
        features.append(np.asarray(temp, dtype=np.float32))
    return np.asarray(features)

In [171]:
h_feat = extract_features(healthy_array)
s_feat = extract_features(schizo_array)

In [174]:
h_feat.shape, s_feat.shape

((6160, 5, 19), (6160, 5, 19))

In [175]:
np.save("h_feat.npy", h_feat)
np.save("s_feat.npy", s_feat)